## connections two containers 

        docker network create todo-app

        docker run -p 5435:5432 \
        --name paw \
        -e 'POSTGRES_PASSWORD=my_password' \
        --network todo-app --network-alias postgres_network \
        -d chriseaton/adventureworks:postgres

        ------------------------------------------------
        docker run -d -p 8888:8888 --name note \
        --network todo-app \
        -e 'POSTGRES_PASSWORD=my_password' \
        -e JUPYTER_TOKEN="easy" \
        -v $HOME/Projects/AIEngineering:/home/jovyan/work/AIEngineering \
        -v $HOME/Projects/pyspark:/home/jovyan/work/pyspark \
        -v $HOME/Projects/datastructure:/home/jovyan/work/datastructure \
        -v $HOME/Projects/pythonclub:/home/jovyan/work/pythonclub \
        -v $HOME/Projects/greatexpectations:/home/jovyan/work/greatexpectations \
        jupyter/all-spark-notebook

        docker logs --tail 3 note

        #ver rede do container
        docker inspect paw | grep IPAddress

### Connect Spark to PostgreSql
        #run into container
        wget https://jdbc.postgresql.org/download/postgresql-42.2.6.jar

In [25]:

# Connect to an existing database
connection = psycopg2.connect(user="postgres",
                                  password="my_password",
                                  host="172.18.0.2",
                                  port="5432",
                                  database="postgres")

In [24]:
from psycopg2 import Error
import psycopg2

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="my_password",
                                  host="172.18.0.2",
                                  port="5432",
                                  database="postgres")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '172.18.0.2', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.7 (Debian 13.7-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',) 

PostgreSQL connection is closed


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "./driver_postgres/postgresql-42.5.1.jar") \
    .getOrCreate()

In [ ]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.18.0.2:5432/postgres") \
    .option("dbtable", "Person") \
    .option("user", "postgres") \
    .option("password", "my_password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [ ]:

df.printSchema()

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql import DataFrameReader

url = 'postgresql://172.18.0.2:5432/postgres'
properties = {'user': 'postgres', 'password': 'my_password'}
df = DataFrameReader(sqlContext).jdbc(
    url='jdbc:%s' % url, table='tablename', properties=properties
)


In [ ]:
df = sqlContext.read.format('jdbc').\
    options(url='jdbc:%s' % url, dbtable='tablename').\
    load()
